
# 🌲 Out-of-Bag (OOB) Evaluation in Random Forest
---

## 1. What is Out-of-Bag (OOB) Evaluation?

**Out-of-Bag (OOB) evaluation** is an *internal validation technique* used in **Random Forest** and **Bagging-based algorithms**.

It allows you to estimate model performance **without creating a separate validation or test set**.

This works because Random Forest uses **bootstrap sampling (sampling with replacement)**.

---

## 2. Bootstrap Sampling Refresher

Assume:
- Dataset size = $N$
- For each tree, we sample $N$ rows **with replacement**

### What happens?

- Some rows are selected **multiple times**
- Some rows are **never selected**

Those never-selected rows are called:

> **Out-of-Bag (OOB) samples**

---

## 3. Why ~37% Data Becomes OOB (Mathematics)

For a given data point $x_i$:

Probability it is **not selected once**:
$$
1 - \frac{1}{N}
$$

Probability it is **never selected in $N$ draws**:
$$
\left(1 - \frac{1}{N}\right)^N
$$

As $N \to \infty$:
$$
\left(1 - \frac{1}{N}\right)^N \approx e^{-1} \approx 0.368
$$

### ✅ Result
$$
\boxed{\text{~36.8\% of data is OOB for each tree}}
$$

---

## 4. How OOB Evaluation Works (Step-by-Step)

1. Train Random Forest with bootstrap sampling
2. For each data point:
   - Identify trees **where this point was NOT used**
3. Predict using **only those trees**
4. Aggregate predictions:
   - Classification → majority vote
   - Regression → mean
5. Compare with true label

This produces the **OOB score**.

---

## 5. Why OOB is Unbiased

- OOB samples were **never seen** during training of those trees
- Equivalent to testing on unseen data
- Statistically similar to cross-validation
- No data leakage

> OOB is a *free validation set* built into Random Forest.

---

## 6. When Should You Use OOB?

### ✅ Best Use Cases
- Small datasets
- When you want quick validation
- Hyperparameter tuning
- Interview explanations

### ❌ Not Ideal When
- Very few trees ($n\_estimators < 50$)
- Bootstrap is disabled
- Extremely imbalanced datasets (use carefully)

---

## 7. Required Hyperparameters for OOB

To enable OOB in scikit-learn:

```python
RandomForestClassifier(
    n_estimators=200,
    bootstrap=True,
    oob_score=True,
    random_state=42
)
```

### Mandatory Conditions
- `bootstrap=True`
- `oob_score=True`
- Sufficient number of trees

---

## 8. Practical Code Example (Classification)

```python
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load data
X, y = load_breast_cancer(return_X_y=True)

# Train-test split (optional, only for comparison)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Random Forest with OOB
rf = RandomForestClassifier(
    n_estimators=200,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

rf.fit(X_train, y_train)

print("OOB Score:", rf.oob_score_)
print("Test Accuracy:", rf.score(X_test, y_test))
```

---

## 9. OOB for Regression

For regression, OOB computes **$R^2$ score**:

```python
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=300,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

rf.fit(X, y)
print("OOB R^2:", rf.oob_score_)
```

---

## 10. OOB vs Train-Test Split

| Aspect | OOB | Train-Test Split |
|------|----|------------------|
| Extra data split | ❌ No | ✅ Yes |
| Data efficiency | High | Lower |
| Bias | Low | Depends |
| Speed | Fast | Medium |
| CV-like | Yes | No |

---

## 11. OOB vs Cross-Validation

- OOB ≈ Leave-one-out behavior
- Faster than k-fold CV
- Slightly higher variance than CV
- Excellent for Random Forest only

---

## 12. Interview-Ready Explanation

> Random Forest uses bootstrap sampling, so around 37% of samples are not used in training each tree. These Out-of-Bag samples act as unseen data. By aggregating predictions from trees where a sample was OOB, we get an unbiased estimate of model performance without a validation set.

---

## 13. Common Mistakes

- Using `oob_score=True` with `bootstrap=False`
- Too few trees
- Comparing OOB directly with training accuracy
- Assuming OOB replaces final test set

---

## 14. Key Takeaways

- OOB is **free validation**
- ~37% data is OOB per tree
- Works only with bootstrap-based models
- Highly interview-relevant
- Great for fast tuning

---

## 15. Final Advice

> Use OOB during **model development**.  
> Still keep a **final test set** for reporting results.



In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=49)

In [14]:
rf = RandomForestClassifier(oob_score=True, random_state=49, max_depth=8, n_estimators=450, bootstrap=True)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=8, n_estimators=450, oob_score=True,
                       random_state=49)

In [15]:
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=8, n_estimators=450, oob_score=True,
                       random_state=49)

In [16]:
rf.oob_score_

0.8347107438016529

In [17]:
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8688524590163934